# 稠密连接网络（DenseNet）

* ResNet significantly changed the view of how to parametrize the functions in deep networks
* *DenseNet* (dense convolutional network) is to some extent the logical extension of this

## 从ResNet到DenseNet

* $H_l(\cdot)$ can be a composite function of operations such as Batch Normalization (BN),
rectified linear units (ReLU), Pooling, or Convolution (Conv)


## 从ResNet到DenseNet

* ResNet $x_l=H_l(x_{l-1})+x_{l-1}$
* DenseNet $x_l=H_l([x_0,x_1,\dotsc,x_{l-1}])$
<center><img src="../img/5_layer_dense_block.png" width="50%"></center>

## 从ResNet到DenseNet

* DenseNets have several compelling advantages: they alleviate the vanishing-gradient
problem, strengthen feature propagation, encourage feature reuse, and substantially reduce the number of parameters
*  explicitly differentiates between information that
is added to the network and information that is preserved

## 从ResNet到DenseNet

* The last layer of such a chain is densely connected to all previous layers

<center><img src="../img/densenet.svg" width="50%"></center>
<center>稠密连接</center>

## 从ResNet到DenseNet

* 稠密网络主要由2部分构成：*稠密块*（dense block）和*过渡层*（transition layer）
* The former define how the inputs and outputs are concatenated, while the latter control the number of channels so that it is not too large

## 稠密块体

DenseNet使用了ResNet改良版的“批量归一化、激活和卷积”架构

In [1]:
import torch
from d2l import torch as d2l
from torch import nn


def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1),
    )

## 稠密块体

* 一个*稠密块*由多个卷积块组成，每个卷积块使用相同数量的输出通道
* 在前向传播中，我们将每个卷积块的输入和输出在通道维上连结

In [2]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            X = torch.cat((X, Y), dim=1)
        return X

## 稠密块体

* 卷积块的通道数控制了输出通道数相对于输入通道数的增长，因此也被称为*增长率*（growth rate）

In [3]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

## 过渡层

* 通过$1\times 1$卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度

In [4]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels),
        nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2),
    )

In [5]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

## DenseNet模型

DenseNet首先使用同ResNet一样的单卷积层和最大池化层

In [6]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
)

## DenseNet模型

* 接下来，类似于ResNet使用的4个残差块，DenseNet使用的是4个稠密块
* 与ResNet类似，可以设置每个稠密块使用多少个卷积层（这里设成4）
* 稠密块里的卷积层通道数（即增长率）设为32，所以每个稠密块将增加128个通道
* 在每个模块之间，ResNet通过步幅为2的残差块减小高和宽，DenseNet则使用过渡层来减半高和宽，并减半通道数

In [7]:
# num_channels为当前的通道数
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间添加一个转换层，使通道数量减半
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

## DenseNet模型

与ResNet类似，最后接上全局池化层和全连接层来输出结果

In [8]:
net = nn.Sequential(
    b1,
    *blks,
    nn.BatchNorm2d(num_channels),
    nn.ReLU(),
    nn.AdaptiveMaxPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10)
)